In [4]:
# STEP 1 — Get a valid gamePk and feed/live link from today's schedule

import requests
from datetime import date

today = date.today().isoformat()
sched_url = "https://statsapi.mlb.com/api/v1/schedule"
params = {"sportId": 1, "date": today}

sched_resp = requests.get(sched_url, params=params)
print("Schedule status:", sched_resp.status_code, "| URL:", sched_resp.url)

sched = sched_resp.json()

# Collect gamePk and link for each game
games_today = []
for g in sched.get("dates", [])[0].get("games", []):
    games_today.append({
        "gamePk": g["gamePk"],
        "feed_link": f"https://statsapi.mlb.com{g['link']}"
    })

print("\nGames found today:")
for game in games_today:
    print(game)

# Pick the first game (or change the index)
game_pk = games_today[0]["gamePk"]
feed_link = games_today[0]["feed_link"]

print("\nUsing gamePk:", game_pk)
print("Feed/live URL:", feed_link)

# STEP 2 — Call /game/{gamePk}/boxscore for that game

box_url = f"https://statsapi.mlb.com/api/v1/game/{game_pk}/boxscore"
response = requests.get(box_url)
print("\nBoxscore status:", response.status_code, "| URL:", response.url)

box = response.json()

# STEP 3 — Inspect the structure of the /game/{gamePk}/boxscore response

# 1. Top-level dictionary keys
print("\nTop-level keys in boxscore response:")
print(list(box.keys()))   # e.g. ['teams', 'officialScorer', 'info', 'pitchingNotes']

# 2. Keys under 'teams'
teams_section = box.get("teams", {})
print("\nKeys inside 'teams':")
print(list(teams_section.keys()))   # should print ['home', 'away']


Schedule status: 200 | URL: https://statsapi.mlb.com/api/v1/schedule?sportId=1&date=2025-09-05

Games found today:
{'gamePk': 776455, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776455/feed/live'}
{'gamePk': 776450, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776450/feed/live'}
{'gamePk': 776452, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776452/feed/live'}
{'gamePk': 776453, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776453/feed/live'}
{'gamePk': 776454, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776454/feed/live'}
{'gamePk': 776457, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776457/feed/live'}
{'gamePk': 776456, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776456/feed/live'}
{'gamePk': 776460, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776460/feed/live'}
{'gamePk': 776451, 'feed_link': 'https://statsapi.mlb.com/api/v1.1/game/776451/feed/live'}
{'gamePk': 776446, 'feed_link': 'https://statsapi.mlb.com/api/v1.1

In [5]:
# STEP — Build a simple scoreboard table from /feed/live

import requests
import pandas as pd

feed_resp = requests.get(feed_link)
print("Feed/live status:", feed_resp.status_code, "| URL:", feed_resp.url)

feed = feed_resp.json()

# Grab core liveData fields
game_data = feed.get("gameData", {})
teams = game_data.get("teams", {})
status = feed.get("gameData", {}).get("status", {})

linescore = feed.get("liveData", {}).get("linescore", {})

row = {
    "gamePk": game_pk,
    "status": status.get("abstractGameState"),      # e.g. "Final", "Live"
    "inning": linescore.get("currentInning"),
    "inningHalf": linescore.get("inningHalf"),
    "outs": linescore.get("outs"),

    "home_team_id": teams.get("home", {}).get("id"),
    "home_team_name": teams.get("home", {}).get("name"),
    "home_score": linescore.get("teams", {}).get("home", {}).get("runs"),

    "away_team_id": teams.get("away", {}).get("id"),
    "away_team_name": teams.get("away", {}).get("name"),
    "away_score": linescore.get("teams", {}).get("away", {}).get("runs"),
}

df_scoreboard = pd.DataFrame([row])
df_scoreboard


Feed/live status: 200 | URL: https://statsapi.mlb.com/api/v1.1/game/776455/feed/live


,gamePk,status,inning,inningHalf,outs,home_team_id,home_team_name,home_score,away_team_id,away_team_name,away_score
0,776455,Final,9,Top,3,112,Chicago Cubs,11,120,Washington Nationals,5


In [6]:
# STEP — Build a full-day scoreboard Silver table (all games)

score_rows = []

for g in games_today:
    gpk = g["gamePk"]
    flink = g["feed_link"]

    resp = requests.get(flink)
    if resp.status_code != 200:
        print("Skipping gamePk", gpk, "status", resp.status_code)
        continue

    feed = resp.json()

    game_data = feed.get("gameData", {})
    teams = game_data.get("teams", {})
    status = game_data.get("status", {})

    linescore = feed.get("liveData", {}).get("linescore", {})

    row = {
        "gamePk": gpk,
        "status": status.get("abstractGameState"),
        "inning": linescore.get("currentInning"),
        "inningHalf": linescore.get("inningHalf"),
        "outs": linescore.get("outs"),

        "home_team_id": teams.get("home", {}).get("id"),
        "home_team_name": teams.get("home", {}).get("name"),
        "home_score": linescore.get("teams", {}).get("home", {}).get("runs"),

        "away_team_id": teams.get("away", {}).get("id"),
        "away_team_name": teams.get("away", {}).get("name"),
        "away_score": linescore.get("teams", {}).get("away", {}).get("runs"),
    }
    score_rows.append(row)

df_scoreboard_all = pd.DataFrame(score_rows)
print("Shape:", df_scoreboard_all.shape)
df_scoreboard_all


Shape: (15, 11)


,gamePk,status,inning,inningHalf,outs,home_team_id,home_team_name,home_score,away_team_id,away_team_name,away_score
0,776455,Final,9,Top,3,112,Chicago Cubs,11,120,Washington Nationals,5
1,776450,Live,2,Bottom,2,113,Cincinnati Reds,0,121,New York Mets,3
2,776452,Live,3,Top,0,116,Detroit Tigers,0,145,Chicago White Sox,1
3,776453,Live,2,Bottom,0,134,Pittsburgh Pirates,0,158,Milwaukee Brewers,1
4,776454,Live,1,Bottom,0,110,Baltimore Orioles,0,119,Los Angeles Dodgers,0
5,776457,Live,1,Top,0,147,New York Yankees,0,141,Toronto Blue Jays,0
6,776456,Live,1,Top,0,146,Miami Marlins,0,143,Philadelphia Phillies,0
7,776460,Live,1,Top,0,144,Atlanta Braves,0,136,Seattle Mariners,0
8,776451,Preview,1,Top,0,139,Tampa Bay Rays,0,114,Cleveland Guardians,0
9,776446,Live,1,Top,0,118,Kansas City Royals,0,142,Minnesota Twins,0
